In [34]:
import requests
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib import style
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

%matplotlib inline
style.use('seaborn')

In [35]:
nba_teams = pd.DataFrame(teams.get_teams())
team_ids = nba_teams['id'].unique()

In [96]:
# main_df.groupby(main_df.SEASON_ID.str[-4:])[['GAME_ID']].count()
#2014 season dates
start_2014 = '2014-10-28'
end_2014 = '2015-04-15'

#2015 season dates
start_2015 = '2015-10-27'
end_2015 = '2016-04-13'

#2016 season dates
start_2016 = '2016-10-25'
end_2016 = '2017-04-12'

#2017 season dates
start_2017 = '2017-10-17'
end_2017 = '2018-04-11'

#2018 season dates
start_2018 = '2018-10-16'
end_2018 = '2019-04-10'

#2019 season dates
start_2019 = '2019-10-22'
end_2019 = '2020-03-11' #ends earlier cos after that we head into the bubble

#2020 season dates
start_2020 = '2020-12-22'

#Repulling the data and computing everything
adjusted_main_df = pd.DataFrame()
for team_id in team_ids:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    games = gamefinder.get_data_frames()[0]
    games = games[(games['GAME_DATE'].between(start_2014,end_2014)) | (games['GAME_DATE'].between(start_2015,end_2015)) | (games['GAME_DATE'].between(start_2016,end_2016)) | (games['GAME_DATE'].between(start_2017,end_2017)) | (games['GAME_DATE'].between(start_2018,end_2018)) | (games['GAME_DATE'].between(start_2019,end_2019)) | (games['GAME_DATE']>start_2020)]
    adjusted_main_df = adjusted_main_df.append(games)
    time.sleep(5)

adjusted_main_df = adjusted_main_df.sort_values('GAME_DATE',ascending=True)
adjusted_main_df['EFG'] = ( adjusted_main_df['FGM'] + 0.5 * adjusted_main_df['FG3M'] ) / adjusted_main_df['FGA']
adjusted_main_df.tail()



,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,EFG
0,22020,1610612760,OKC,Oklahoma City Thunder,0022000251,2021-01-24,OKC @ LAC,L,241,100,...,12.0,39.0,51.0,22,7.0,2,10,19,-8.0,0.478947
0,22020,1610612746,LAC,LA Clippers,0022000251,2021-01-24,LAC vs. OKC,W,238,108,...,8.0,38.0,46.0,26,4.0,2,10,10,8.0,0.521739
0,22020,1610612761,TOR,Toronto Raptors,0022000249,2021-01-24,TOR @ IND,W,240,107,...,12.0,33.0,45.0,24,12.0,8,16,18,5.0,0.494444
0,22020,1610612752,NYK,New York Knicks,0022000256,2021-01-24,NYK @ POR,L,241,113,...,13.0,34.0,47.0,20,9.0,0,12,20,-3.0,0.523256
0,22020,1610612737,ATL,Atlanta Hawks,0022000254,2021-01-24,ATL @ MIL,L,240,115,...,5.0,22.0,27.0,27,8.0,4,9,23,-14.0,0.590361


In [97]:

master_data = adjusted_main_df

Create a team dictionary to store all the different statistics for all teams

In [98]:
nba_teams = {
    'ATL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'BKN': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'BOS': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'CHA': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'CHI': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'CLE': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'DAL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'DEN': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'DET': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'GSW': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'HOU': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'IND': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'LAC': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'LAL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'MEM': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'MIA': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'MIL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'MIN': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'NOP': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'NYK': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'OKC': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'ORL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'PHI': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'PHX': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'POR': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'SAC': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'SAS': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'TOR': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'UTA': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB': 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'WAS': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DREB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 }
    
}

Determining the columns 

In [99]:
master_data= master_data.sort_values(by = "GAME_DATE", ascending = True)
master_data.index = range(len(master_data))
master_data.head()
master_data.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'EFG'],
      dtype='object')

combining the opponent team and home team data set onto the same row

In [100]:
master_data['HOME_COURT'] = np.where(master_data['MATCHUP'].str.contains('vs.'), 'H', 'A')
master_data['BEF_AVG_PTS'] = 0
master_data['BEF_AVG_AST'] = 0
master_data['BEF_AVG_OREB'] = 0
master_data['BEF_AVG_DREB'] = 0
master_data['AFT_AVG_PTS'] = 0
master_data['AFT_AVG_AST'] = 0
master_data['AFT_AVG_OREB'] = 0
master_data['AFT_AVG_DREB'] = 0
master_data['GAME_PLAYED'] = 0
master_data['cPTS'] = 0
master_data['cAST'] = 0
master_data['cOREB'] = 0
master_data['cDREB'] = 0
master_data['cFGA'] = 0
master_data['cTO'] = 0
master_data['cFTA'] = 0

master_data['DIS_PTS'] = 0
master_data['DIS_AST'] = 0
master_data['DIS_OREB'] = 0
master_data['DIS_DREB'] = 0

master_data["OFFRATE"] = 0
master_data['DEFRATE'] = 0


master_data.head()
master_data_combined = master_data.merge(master_data, on='GAME_ID')
master_data_combined = master_data_combined.drop(master_data_combined[master_data_combined['TEAM_ID_x'] == master_data_combined['TEAM_ID_y']].index)
master_data_combined = master_data_combined.iloc[::2]

master_data_combined.tail()

,SEASON_ID_x,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS_x,...,cDREB_y,cFGA_y,cTO_y,cFTA_y,DIS_PTS_y,DIS_AST_y,DIS_OREB_y,DIS_DREB_y,OFFRATE_y,DEFRATE_y
29405,22020,1610612764,WAS,Washington Wizards,0022000253,2021-01-24,WAS @ SAS,L,240,101,...,0,0,0,0,0,0,0,0,0,0
29409,22020,1610612749,MIL,Milwaukee Bucks,0022000254,2021-01-24,MIL vs. ATL,W,240,129,...,0,0,0,0,0,0,0,0,0,0
29413,22020,1610612766,CHA,Charlotte Hornets,0022000252,2021-01-24,CHA @ ORL,W,241,107,...,0,0,0,0,0,0,0,0,0,0
29417,22020,1610612757,POR,Portland Trail Blazers,0022000256,2021-01-24,POR vs. NYK,W,240,116,...,0,0,0,0,0,0,0,0,0,0
29421,22020,1610612738,BOS,Boston Celtics,0022000250,2021-01-24,BOS vs. CLE,W,239,141,...,0,0,0,0,0,0,0,0,0,0


Separate into different seasons 

In [101]:
season_2014 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22014']
season_2014.index = range(len(season_2014))

season_2015 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22015']
season_2015.index = range(len(season_2015))

season_2016 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22016']
season_2016.index = range(len(season_2016))

season_2017 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22017']
season_2017.index = range(len(season_2017))


season_2018 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22018']
season_2018.index = range(len(season_2018))

season_2019 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22019']
season_2019.index = range(len(season_2019))

season_2020 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22020']
season_2020.index = range(len(season_2020))

# {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
#             'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
#            'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0
#            "cFTA":0 }

Function used to append stats into dataset

In [20]:
def add_into_dataset(i,season):
    # #     add in the data into the dataset 
    
    season.loc[i,'BEF_AVG_PTS_x'] = nba_teams[team_1]['BEF_AVG_PTS']
    season.loc[i,'BEF_AVG_PTS_y'] = nba_teams[team_2]['BEF_AVG_PTS']
    season.loc[i,'BEF_AVG_AST_x'] = nba_teams[team_1]['BEF_AVG_AST']
    season.loc[i,'BEF_AVG_AST_y'] = nba_teams[team_2]['BEF_AVG_AST']
    season.loc[i,'BEF_AVG_OREB_x'] = nba_teams[team_1]['BEF_AVG_OREB']
    season.loc[i,'BEF_AVG_OREB_y'] = nba_teams[team_2]['BEF_AVG_OREB']
    season.loc[i,'BEF_AVG_DREB_x'] = nba_teams[team_1]['BEF_AVG_DREB']
    season.loc[i,'BEF_AVG_DREB_y'] = nba_teams[team_2]['BEF_AVG_DREB']
    
    season.loc[i,'AFT_AVG_PTS_x'] = nba_teams[team_1]['AFT_AVG_PTS']
    season.loc[i,'AFT_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
    season.loc[i,'AFT_AVG_AST_x'] = nba_teams[team_1]['AFT_AVG_AST']
    season.loc[i,'AFT_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
    season.loc[i,'AFT_AVG_OREB_x'] = nba_teams[team_1]['AFT_AVG_OREB']
    season.loc[i,'AFT_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
    season.loc[i,'AFT_AVG_DREB_x'] = nba_teams[team_1]['AFT_AVG_DREB']
    season.loc[i,'AFT_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
    
    season.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
    season.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED']                                                      
    season.loc[i,'cPTS_x'] = nba_teams[team_1]['cPTS']
    season.loc[i,'cAST_x'] = nba_teams[team_1]['cAST']
    season.loc[i,'cOREB_x'] = nba_teams[team_1]['cOREB']
    season.loc[i,'cDREB_x'] = nba_teams[team_1]['cDREB']
    season.loc[i,'cFGA_x'] = nba_teams[team_1]['cFGA']
    season.loc[i,'cTO_x'] = nba_teams[team_1]['cTO']
    season.loc[i,'cFTA_x'] = nba_teams[team_1]['cFTA']
    season.loc[i,'cPTS_y'] = nba_teams[team_2]['cPTS']
    season.loc[i,'cAST_y'] = nba_teams[team_2]['cAST']
    season.loc[i,'cOREB_y'] = nba_teams[team_2]['cOREB']
    season.loc[i,'cDREB_y'] = nba_teams[team_2]['cDREB']
    season.loc[i,'cFGA_y'] = nba_teams[team_2]['cFGA']
    season.loc[i,'cTO_y'] = nba_teams[team_2]['cTO']
    season.loc[i,'cFTA_y'] = nba_teams[team_2]['cFTA']

    season.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
    season.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
    season.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
    season.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']
    season.loc[i,'DIS_PTS_y'] = nba_teams[team_2]['DIS_PTS']
    season.loc[i,'DIS_AST_y'] = nba_teams[team_2]['DIS_AST']
    season.loc[i,'DIS_OREB_y'] = nba_teams[team_2]['DIS_OREB']
    season.loc[i,'DIS_DREB_y'] = nba_teams[team_2]['DIS_DREB']

    season.loc[i,"OFFRATE_x"] = nba_teams[team_1]['OFFRATE']
    season.loc[i,'DEFRATE_x'] = nba_teams[team_1]['DEFRATE']
    season.loc[i,"OFFRATE_y"] = nba_teams[team_2]['OFFRATE']
    season.loc[i,'DEFRATE_y'] = nba_teams[team_2]['DEFRATE']
    
#     print('done')

In [ ]:
# for i, row in season_2014.iterrows():
# #         get the team name
#     team_1 = row['TEAM_ABBREVIATION_x']
#     team_2 = row['TEAM_ABBREVIATION_y']

# #     print(nba_teams[team_1]['GAME_PLAYED'] )
    
# #     update team 1 and 2
#     if(nba_teams[team_1]['GAME_PLAYED'] == 0):
# # #         team 1
#         nba_teams[team_1]['GAME_PLAYED'] += 1
#         nba_teams[team_1]['cPTS'] += row['PTS_x']
#         nba_teams[team_1]['cAST'] += row['AST_x']
#         nba_teams[team_1]['cOREB'] += row['OREB_x']
#         nba_teams[team_1]['cDREB'] += row['DREB_x']
#         nba_teams[team_1]['cFGA'] += row['FGA_x']
#         nba_teams[team_1]['cTO'] += row['TOV_x']
#         nba_teams[team_1]['cFTA'] += row['FTA_x']
        

#         nba_teams[team_1]['AFT_AVG_PTS'] = nba_teams[team_1]['cPTS']
#         nba_teams[team_1]['AFT_AVG_AST'] = nba_teams[team_1]['cAST']
#         nba_teams[team_1]['AFT_AVG_OREB'] = nba_teams[team_1]['cOREB']
#         nba_teams[team_1]['AFT_AVG_DREB'] = nba_teams[team_1]['cDREB']
        
#         #         calculating off def ratings for team 1 
#         tot_pos_1 = nba_teams[team_1]['cFGA'] - nba_teams[team_1]['cOREB'] + nba_teams[team_1]['cTO'] +(0.4* nba_teams[team_1]['cFTA'])
#         off_ratings_1 = nba_teams[team_1]['cPTS']/tot_pos_1
#         nba_teams[team_1]['OFFRATE'] = off_ratings_1
        
#         def_ratings_1 = nba_teams[team_2]['cPTS']/tot_pos_1
#         nba_teams[team_1]['DEFRATE'] = def_ratings_1
        
# #         DISPARITY BETWEEN 2 TEAMS 
#         nba_teams[team_1]['DIS_PTS'] = nba_teams[team_1]['BEF_AVG_PTS'] - nba_teams[team_2]['BEF_AVG_PTS']
#         nba_teams[team_1]['DIS_AST'] = nba_teams[team_1]['BEF_AVG_AST'] - nba_teams[team_2]['BEF_AVG_AST']
#         nba_teams[team_1]['DIS_OREB'] = nba_teams[team_1]['BEF_AVG_OREB'] - nba_teams[team_2]['BEF_AVG_OREB']
#         nba_teams[team_1]['DIS_DREB'] = nba_teams[team_1]['BEF_AVG_DREB'] - nba_teams[team_2]['BEF_AVG_DREB']
        
#     elif(nba_teams[team_1]['GAME_PLAYED'] != 0):
# #         add a before avg ratings for the previous gams 
#         nba_teams[team_1]['BEF_AVG_PTS'] = nba_teams[team_1]['AFT_AVG_PTS']
#         nba_teams[team_1]['BEF_AVG_AST'] = nba_teams[team_1]['AFT_AVG_AST']
#         nba_teams[team_1]['BEF_AVG_OREB'] = nba_teams[team_1]['AFT_AVG_OREB']
#         nba_teams[team_1]['BEF_AVG_DREB'] = nba_teams[team_1]['AFT_AVG_DREB']
        
    
#         nba_teams[team_1]['GAME_PLAYED'] += 1
#         nba_teams[team_1]['cPTS'] += row['PTS_x']
#         nba_teams[team_1]['cAST'] += row['AST_x']
#         nba_teams[team_1]['cOREB'] += row['OREB_x']
#         nba_teams[team_1]['cDREB'] += row['DREB_x']

#         nba_teams[team_1]['AFT_AVG_PTS'] = nba_teams[team_1]['cPTS']/nba_teams[team_1]['GAME_PLAYED']
#         nba_teams[team_1]['AFT_AVG_AST'] = nba_teams[team_1]['cAST']/nba_teams[team_1]['GAME_PLAYED']
#         nba_teams[team_1]['AFT_AVG_OREB'] = nba_teams[team_1]['cOREB']/nba_teams[team_1]['GAME_PLAYED']
#         nba_teams[team_1]['AFT_AVG_DREB'] = nba_teams[team_1]['cDREB']/nba_teams[team_1]['GAME_PLAYED']
        
#          #         calculating off def ratings for team 1 
#         tot_pos_1 = nba_teams[team_1]['cFGA'] - nba_teams[team_1]['cOREB'] + nba_teams[team_1]['cTO'] +(0.4* nba_teams[team_1]['cFTA'])
#         off_ratings_1 = nba_teams[team_1]['cPTS']/tot_pos_1
#         nba_teams[team_1]['OFFRATE'] = off_ratings_1
        
#         def_ratings_1 = nba_teams[team_2]['cPTS']/tot_pos_1
#         nba_teams[team_1]['DEFRATE'] = def_ratings_1
#         #         DISPARITY BETWEEN 2 TEAMS 
#         nba_teams[team_1]['DIS_PTS'] = nba_teams[team_1]['BEF_AVG_PTS'] - nba_teams[team_2]['BEF_AVG_PTS']
#         nba_teams[team_1]['DIS_AST'] = nba_teams[team_1]['BEF_AVG_AST'] - nba_teams[team_2]['BEF_AVG_AST']
#         nba_teams[team_1]['DIS_OREB'] = nba_teams[team_1]['BEF_AVG_OREB'] - nba_teams[team_2]['BEF_AVG_OREB']
#         nba_teams[team_1]['DIS_DREB'] = nba_teams[team_1]['BEF_AVG_DREB'] - nba_teams[team_2]['BEF_AVG_DREB']
        
#     if(nba_teams[team_2]['GAME_PLAYED'] == 0):
        
# #         team 2 
#         nba_teams[team_2]['GAME_PLAYED'] += 1
#         nba_teams[team_2]['cPTS'] += row['PTS_y']
#         nba_teams[team_2]['cAST'] += row['AST_y']
#         nba_teams[team_2]['cOREB'] += row['OREB_y']
#         nba_teams[team_2]['cDREB'] += row['DREB_y']
#         nba_teams[team_2]['cFGA'] += row['FGA_y']
#         nba_teams[team_2]['cTO'] += row['TOV_y']
#         nba_teams[team_2]['cFTA'] += row['FTA_y']
        

#         nba_teams[team_2]['AFT_AVG_PTS'] = nba_teams[team_2]['cPTS']
#         nba_teams[team_2]['AFT_AVG_AST'] = nba_teams[team_2]['cAST']
#         nba_teams[team_2]['AFT_AVG_OREB'] = nba_teams[team_2]['cOREB']
#         nba_teams[team_2]['AFT_AVG_DREB'] = nba_teams[team_2]['cDREB']
        
        
#         #         calculating off def ratings for team 2
#         tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
#         off_ratings_2 = nba_teams[team_2]['cPTS']/tot_pos_2
#         nba_teams[team_2]['OFFRATE'] = off_ratings_2
        
#         def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
#         nba_teams[team_2]['DEFRATE'] = def_ratings_2
        
#         #         DISPARITY BETWEEN 2 TEAMS 
#         nba_teams[team_2]['DIS_PTS'] = nba_teams[team_2]['BEF_AVG_PTS'] - nba_teams[team_1]['BEF_AVG_PTS']
#         nba_teams[team_2]['DIS_AST'] = nba_teams[team_2]['BEF_AVG_AST'] - nba_teams[team_1]['BEF_AVG_AST']
#         nba_teams[team_2]['DIS_OREB'] = nba_teams[team_2]['BEF_AVG_OREB'] - nba_teams[team_1]['BEF_AVG_OREB']
#         nba_teams[team_2]['DIS_DREB'] = nba_teams[team_2]['BEF_AVG_DREB'] - nba_teams[team_1]['BEF_AVG_DREB']
    

        
        
#     elif(nba_teams[team_2]['GAME_PLAYED'] != 0):
# #         add a before avg ratings for the previous gams 
#         nba_teams[team_2]['BEF_AVG_PTS'] = nba_teams[team_2]['AFT_AVG_PTS']
#         nba_teams[team_2]['BEF_AVG_AST'] = nba_teams[team_2]['AFT_AVG_AST']
#         nba_teams[team_2]['BEF_AVG_OREB'] = nba_teams[team_2]['AFT_AVG_OREB']
#         nba_teams[team_2]['BEF_AVG_DREB'] = nba_teams[team_2]['AFT_AVG_DREB']
        
    
#         nba_teams[team_2]['GAME_PLAYED'] += 1
#         nba_teams[team_2]['cPTS'] += row['PTS_y']
#         nba_teams[team_2]['cAST'] += row['AST_y']
#         nba_teams[team_2]['cOREB'] += row['OREB_y']
#         nba_teams[team_2]['cDREB'] += row['DREB_y']

#         nba_teams[team_2]['AFT_AVG_PTS'] = nba_teams[team_2]['cPTS']/nba_teams[team_2]['GAME_PLAYED']
#         nba_teams[team_2]['AFT_AVG_AST'] = nba_teams[team_2]['cAST']/nba_teams[team_2]['GAME_PLAYED']
#         nba_teams[team_2]['AFT_AVG_OREB'] = nba_teams[team_2]['cOREB']/nba_teams[team_2]['GAME_PLAYED']
#         nba_teams[team_2]['AFT_AVG_DREB'] = nba_teams[team_2]['cDREB']/nba_teams[team_2]['GAME_PLAYED']
        
#          #         calculating off def ratings for team 2
#         tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
#         off_ratings_2 = nba_teams[team_2]['cPTS']/tot_pos_2
#         nba_teams[team_2]['OFFRATE'] = off_ratings_2
        
#         def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
#         nba_teams[team_2]['DEFRATE'] = def_ratings_2
#                 #         DISPARITY BETWEEN 2 TEAMS 
#         nba_teams[team_2]['DIS_PTS'] = nba_teams[team_2]['BEF_AVG_PTS'] - nba_teams[team_1]['BEF_AVG_PTS']
#         nba_teams[team_2]['DIS_AST'] = nba_teams[team_2]['BEF_AVG_AST'] - nba_teams[team_1]['BEF_AVG_AST']
#         nba_teams[team_2]['DIS_OREB'] = nba_teams[team_2]['BEF_AVG_OREB'] - nba_teams[team_1]['BEF_AVG_OREB']
#         nba_teams[team_2]['DIS_DREB'] = nba_teams[team_2]['BEF_AVG_DREB'] - nba_teams[team_1]['BEF_AVG_DREB']
    
# # #     add in the data into the dataset 
#         add_into_dataset(season_2014)
# #     season_2014.loc[i,'BEF_AVG_PTS_x'] = nba_teams[team_1]['BEF_AVG_PTS']
# #     season_2014.loc[i,'BEF_AVG_PTS_y'] = nba_teams[team_2]['BEF_AVG_PTS']
# #     season_2014.loc[i,'BEF_AVG_AST_x'] = nba_teams[team_1]['BEF_AVG_AST']
# #     season_2014.loc[i,'BEF_AVG_AST_y'] = nba_teams[team_2]['BEF_AVG_AST']
# #     season_2014.loc[i,'BEF_AVG_OREB_x'] = nba_teams[team_1]['BEF_AVG_OREB']
# #     season_2014.loc[i,'BEF_AVG_OREB_y'] = nba_teams[team_2]['BEF_AVG_OREB']
# #     season_2014.loc[i,'BEF_AVG_DREB_x'] = nba_teams[team_1]['BEF_AVG_DREB']
# #     season_2014.loc[i,'BEF_AVG_DREB_y'] = nba_teams[team_2]['BEF_AVG_DREB']
    
# #     season_2014.loc[i,'AFT_AVG_PTS_x'] = nba_teams[team_1]['AFT_AVG_PTS']
# #     season_2014.loc[i,'AFT_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
# #     season_2014.loc[i,'AFT_AVG_AST_x'] = nba_teams[team_1]['AFT_AVG_AST']
# #     season_2014.loc[i,'AFT_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
# #     season_2014.loc[i,'AFT_AVG_OREB_x'] = nba_teams[team_1]['AFT_AVG_OREB']
# #     season_2014.loc[i,'AFT_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
# #     season_2014.loc[i,'AFT_AVG_DREB_x'] = nba_teams[team_1]['AFT_AVG_DREB']
# #     season_2014.loc[i,'AFT_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
    
# #     season_2014.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
# #     season_2014.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED']                                                      
# #     season_2014.loc[i,'cPTS_x'] = nba_teams[team_1]['cPTS']
# #     season_2014.loc[i,'cAST_x'] = nba_teams[team_1]['cAST']
# #     season_2014.loc[i,'cOREB_x'] = nba_teams[team_1]['cOREB']
# #     season_2014.loc[i,'cDREB_x'] = nba_teams[team_1]['cDREB']
# #     season_2014.loc[i,'cFGA_x'] = nba_teams[team_1]['cFGA']
# #     season_2014.loc[i,'cTO_x'] = nba_teams[team_1]['cTO']
# #     season_2014.loc[i,'cFTA_x'] = nba_teams[team_1]['cFTA']
# #     season_2014.loc[i,'cPTS_y'] = nba_teams[team_2]['cPTS']
# #     season_2014.loc[i,'cAST_y'] = nba_teams[team_2]['cAST']
# #     season_2014.loc[i,'cOREB_y'] = nba_teams[team_2]['cOREB']
# #     season_2014.loc[i,'cDREB_y'] = nba_teams[team_2]['cDREB']
# #     season_2014.loc[i,'cFGA_y'] = nba_teams[team_2]['cFGA']
# #     season_2014.loc[i,'cTO_y'] = nba_teams[team_2]['cTO']
# #     season_2014.loc[i,'cFTA_y'] = nba_teams[team_2]['cFTA']

# #     season_2014.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
# #     season_2014.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
# #     season_2014.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
# #     season_2014.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']
# #     season_2014.loc[i,'DIS_PTS_y'] = nba_teams[team_2]['DIS_PTS']
# #     season_2014.loc[i,'DIS_AST_y'] = nba_teams[team_2]['DIS_AST']
# #     season_2014.loc[i,'DIS_OREB_y'] = nba_teams[team_2]['DIS_OREB']
# #     season_2014.loc[i,'DIS_DREB_y'] = nba_teams[team_2]['DIS_DREB']

# #     season_2014.loc[i,"OFFRATE_x"] = nba_teams[team_1]['OFFRATE']
# #     season_2014.loc[i,'DEFRATE_x'] = nba_teams[team_1]['DEFRATE']
# #     season_2014.loc[i,"OFFRATE_y"] = nba_teams[team_2]['OFFRATE']
# #     season_2014.loc[i,'DEFRATE_y'] = nba_teams[team_2]['DEFRATE']                                                            
    
        
        

In [95]:
print(season_2020['GAME_PLAYED_x'])



0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     2
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
205    0
206    0
207    0
208    0
209    0
210    0
211    0
212    0
213    0
214    0
215    0
216    0
217    0
218    0
219    0
220    0
221    0
222    0
223    0
224    0
225    0
226    0
227    0
228    0
229    0
230    0
231    0
232    0
233    0
234    0
Name: GAME_PLAYED_x, Length: 235, dtype: int64


In [109]:
for i, row in season_2020.iterrows():
#         get the team name
    team_1 = row['TEAM_ABBREVIATION_x']
    team_2 = row['TEAM_ABBREVIATION_y']

#     update team 1 and 2
    if(row['GAME_PLAYED_x'] == 0):
#         print('hi')
# #         team 1
    
        nba_teams[team_1]['GAME_PLAYED'] += 1
        season_2020.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
        
        nba_teams[team_1]['BEF_AVG_PTS'] = 0
        nba_teams[team_1]['BEF_AVG_AST'] = 0
        nba_teams[team_1]['BEF_AVG_OREB'] = 0
        nba_teams[team_1]['BEF_AVG_DREB'] = 0

# # appending the statistics        
        season_2020.loc[i,'BEF_AVG_PTS_x'] = nba_teams[team_1]['BEF_AVG_PTS']
        season_2020.loc[i,'BEF_AVG_AST_x'] = nba_teams[team_1]['BEF_AVG_AST']
        season_2020.loc[i,'BEF_AVG_OREB_x'] = nba_teams[team_1]['BEF_AVG_OREB']
        season_2020.loc[i,'BEF_AVG_DREB_x'] = nba_teams[team_1]['BEF_AVG_DREB']
   
        
        
        nba_teams[team_1]['cPTS'] += row['PTS_x']
        nba_teams[team_1]['cAST'] += row['AST_x']
        nba_teams[team_1]['cOREB'] += row['OREB_x']
        nba_teams[team_1]['cDREB'] += row['DREB_x']
        nba_teams[team_1]['cFGA'] += row['FGA_x']
        nba_teams[team_1]['cTO'] += row['TOV_x']
        nba_teams[team_1]['cFTA'] += row['FTA_x']
        
#         # appending the statistics
        
        season_2020.loc[i,'cPTS_x'] = nba_teams[team_1]['cPTS']
        season_2020.loc[i,'cAST_x'] = nba_teams[team_1]['cAST']
        season_2020.loc[i,'cOREB_x'] = nba_teams[team_1]['cOREB']
        season_2020.loc[i,'cDREB_x'] = nba_teams[team_1]['cDREB']
        season_2020.loc[i,'cFGA_x'] = nba_teams[team_1]['cFGA']
        season_2020.loc[i,'cTO_x'] = nba_teams[team_1]['cTO']
        season_2020.loc[i,'cFTA_x'] = nba_teams[team_1]['cFTA']
        

        nba_teams[team_1]['AFT_AVG_PTS'] = nba_teams[team_1]['cPTS']
        nba_teams[team_1]['AFT_AVG_AST'] = nba_teams[team_1]['cAST']
        nba_teams[team_1]['AFT_AVG_OREB'] = nba_teams[team_1]['cOREB']
        nba_teams[team_1]['AFT_AVG_DREB'] = nba_teams[team_1]['cDREB']
        
#         # appending the statistics
        
        season_2020.loc[i,'AFT_AVG_PTS_x'] = nba_teams[team_1]['AFT_AVG_PTS']
        season_2020.loc[i,'AFT_AVG_AST_x'] = nba_teams[team_1]['AFT_AVG_AST']
        season_2020.loc[i,'AFT_AVG_OREB_x'] = nba_teams[team_1]['AFT_AVG_OREB']
        season_2020.loc[i,'AFT_AVG_DREB_x'] = nba_teams[team_1]['AFT_AVG_DREB']

        #         calculating off def ratings for team 1 
        tot_pos_1 = nba_teams[team_1]['cFGA'] - nba_teams[team_1]['cOREB'] + nba_teams[team_1]['cTO'] +(0.4* nba_teams[team_1]['cFTA'])
        off_ratings_1 = nba_teams[team_1]['cPTS']/tot_pos_1
        nba_teams[team_1]['OFFRATE'] = off_ratings_1
        
        def_ratings_1 = nba_teams[team_2]['cPTS']/tot_pos_1
        nba_teams[team_1]['DEFRATE'] = def_ratings_1
        # appending the statistics
        season_2020.loc[i,"OFFRATE_x"] = nba_teams[team_1]['OFFRATE']
        season_2020.loc[i,'DEFRATE_x'] = nba_teams[team_1]['DEFRATE']
#         DISPARITY BETWEEN 2 TEAMS 
        nba_teams[team_1]['DIS_PTS'] = nba_teams[team_1]['BEF_AVG_PTS'] - nba_teams[team_2]['BEF_AVG_PTS']
        nba_teams[team_1]['DIS_AST'] = nba_teams[team_1]['BEF_AVG_AST'] - nba_teams[team_2]['BEF_AVG_AST']
        nba_teams[team_1]['DIS_OREB'] = nba_teams[team_1]['BEF_AVG_OREB'] - nba_teams[team_2]['BEF_AVG_OREB']
        nba_teams[team_1]['DIS_DREB'] = nba_teams[team_1]['BEF_AVG_DREB'] - nba_teams[team_2]['BEF_AVG_DREB']
        # appending the statistics
        season_2020.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
        season_2020.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
        season_2020.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
        season_2020.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']
        
#     else:
# #         add a before avg ratings for the previous gams 
#         nba_teams[team_1]['BEF_AVG_PTS'] = nba_teams[team_1]['AFT_AVG_PTS']
#         nba_teams[team_1]['BEF_AVG_AST'] = nba_teams[team_1]['AFT_AVG_AST']
#         nba_teams[team_1]['BEF_AVG_OREB'] = nba_teams[team_1]['AFT_AVG_OREB']
#         nba_teams[team_1]['BEF_AVG_DREB'] = nba_teams[team_1]['AFT_AVG_DREB']
#         # appending the statistics
#         season_2020.loc[i,'BEF_AVG_PTS_x'] = nba_teams[team_1]['BEF_AVG_PTS']
#         season_2020.loc[i,'BEF_AVG_AST_x'] = nba_teams[team_1]['BEF_AVG_AST']
#         season_2020.loc[i,'BEF_AVG_OREB_x'] = nba_teams[team_1]['BEF_AVG_OREB']
#         season_2020.loc[i,'BEF_AVG_DREB_x'] = nba_teams[team_1]['BEF_AVG_DREB']
   
#         # appending the statistics
    
#         nba_teams[team_1]['GAME_PLAYED'] += 1
#         season_2020.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
        
#         nba_teams[team_1]['cPTS'] += row['PTS_x']
#         nba_teams[team_1]['cAST'] += row['AST_x']
#         nba_teams[team_1]['cOREB'] += row['OREB_x']
#         nba_teams[team_1]['cDREB'] += row['DREB_x']
#         nba_teams[team_1]['cFGA'] += row['FGA_x']
#         nba_teams[team_1]['cTO'] += row['TOV_x']
#         nba_teams[team_1]['cFTA'] += row['FTA_x']
#         # appending the statistics
#         season_2020.loc[i,'cPTS_x'] = nba_teams[team_1]['cPTS']
#         season_2020.loc[i,'cAST_x'] = nba_teams[team_1]['cAST']
#         season_2020.loc[i,'cOREB_x'] = nba_teams[team_1]['cOREB']
#         season_2020.loc[i,'cDREB_x'] = nba_teams[team_1]['cDREB']
#         season_2020.loc[i,'cFGA_x'] = nba_teams[team_1]['cFGA']
#         season_2020.loc[i,'cTO_x'] = nba_teams[team_1]['cTO']
#         season_2020.loc[i,'cFTA_x'] = nba_teams[team_1]['cFTA']

#         nba_teams[team_1]['AFT_AVG_PTS'] = nba_teams[team_1]['cPTS']/nba_teams[team_1]['GAME_PLAYED']
#         nba_teams[team_1]['AFT_AVG_AST'] = nba_teams[team_1]['cAST']/nba_teams[team_1]['GAME_PLAYED']
#         nba_teams[team_1]['AFT_AVG_OREB'] = nba_teams[team_1]['cOREB']/nba_teams[team_1]['GAME_PLAYED']
#         nba_teams[team_1]['AFT_AVG_DREB'] = nba_teams[team_1]['cDREB']/nba_teams[team_1]['GAME_PLAYED']
#         # appending the statistics
#         season_2020.loc[i,'AFT_AVG_PTS_x'] = nba_teams[team_1]['AFT_AVG_PTS']
#         season_2020.loc[i,'AFT_AVG_AST_x'] = nba_teams[team_1]['AFT_AVG_AST']
#         season_2020.loc[i,'AFT_AVG_OREB_x'] = nba_teams[team_1]['AFT_AVG_OREB']
#         season_2020.loc[i,'AFT_AVG_DREB_x'] = nba_teams[team_1]['AFT_AVG_DREB']
        
#          #         calculating off def ratings for team 1 
#         tot_pos_1 = nba_teams[team_1]['cFGA'] - nba_teams[team_1]['cOREB'] + nba_teams[team_1]['cTO'] +(0.4* nba_teams[team_1]['cFTA'])
#         off_ratings_1 = nba_teams[team_1]['cPTS']/tot_pos_1
#         nba_teams[team_1]['OFFRATE'] = off_ratings_1
        
#         def_ratings_1 = nba_teams[team_2]['cPTS']/tot_pos_1
#         nba_teams[team_1]['DEFRATE'] = def_ratings_1
#         # appending the statistics
#         season_2020.loc[i,"OFFRATE_x"] = nba_teams[team_1]['OFFRATE']
#         season_2020.loc[i,'DEFRATE_x'] = nba_teams[team_1]['DEFRATE']
#         #         DISPARITY BETWEEN 2 TEAMS 
#         nba_teams[team_1]['DIS_PTS'] = nba_teams[team_1]['BEF_AVG_PTS'] - nba_teams[team_2]['BEF_AVG_PTS']
#         nba_teams[team_1]['DIS_AST'] = nba_teams[team_1]['BEF_AVG_AST'] - nba_teams[team_2]['BEF_AVG_AST']
#         nba_teams[team_1]['DIS_OREB'] = nba_teams[team_1]['BEF_AVG_OREB'] - nba_teams[team_2]['BEF_AVG_OREB']
#         nba_teams[team_1]['DIS_DREB'] = nba_teams[team_1]['BEF_AVG_DREB'] - nba_teams[team_2]['BEF_AVG_DREB']
#         # appending the statistics
#         season_2020.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
#         season_2020.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
#         season_2020.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
#         season_2020.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']
        
#     if(row['GAME_PLAYED_y'] == 0):
        
# #         team 2 
#         nba_teams[team_2]['GAME_PLAYED'] += 1
#         season_2020.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED']  
        
#         nba_teams[team_1]['BEF_AVG_PTS'] = 0
#         nba_teams[team_1]['BEF_AVG_AST'] = 0
#         nba_teams[team_1]['BEF_AVG_OREB'] = 0
#         nba_teams[team_1]['BEF_AVG_DREB'] = 0
#         # appending the statistics
#         season_2020.loc[i,'BEF_AVG_PTS_y'] = nba_teams[team_2]['BEF_AVG_PTS']
#         season_2020.loc[i,'BEF_AVG_AST_y'] = nba_teams[team_2]['BEF_AVG_AST']
#         season_2020.loc[i,'BEF_AVG_OREB_y'] = nba_teams[team_2]['BEF_AVG_OREB']
#         season_2020.loc[i,'BEF_AVG_DREB_y'] = nba_teams[team_2]['BEF_AVG_DREB']
        
#         nba_teams[team_2]['cPTS'] = row['PTS_y']
#         nba_teams[team_2]['cAST'] = row['AST_y']
#         nba_teams[team_2]['cOREB'] = row['OREB_y']
#         nba_teams[team_2]['cDREB'] = row['DREB_y']
#         nba_teams[team_2]['cFGA'] = row['FGA_y']
#         nba_teams[team_2]['cTO'] = row['TOV_y']
#         nba_teams[team_2]['cFTA'] = row['FTA_y']
#         # appending the statistics
#         season_2020.loc[i,'cPTS_y'] = nba_teams[team_2]['cPTS']
#         season_2020.loc[i,'cAST_y'] = nba_teams[team_2]['cAST']
#         season_2020.loc[i,'cOREB_y'] = nba_teams[team_2]['cOREB']
#         season_2020.loc[i,'cDREB_y'] = nba_teams[team_2]['cDREB']
#         season_2020.loc[i,'cFGA_y'] = nba_teams[team_2]['cFGA']
#         season_2020.loc[i,'cTO_y'] = nba_teams[team_2]['cTO']
#         season_2020.loc[i,'cFTA_y'] = nba_teams[team_2]['cFTA']
        

#         nba_teams[team_2]['AFT_AVG_PTS'] = nba_teams[team_2]['cPTS']
#         nba_teams[team_2]['AFT_AVG_AST'] = nba_teams[team_2]['cAST']
#         nba_teams[team_2]['AFT_AVG_OREB'] = nba_teams[team_2]['cOREB']
#         nba_teams[team_2]['AFT_AVG_DREB'] = nba_teams[team_2]['cDREB']
#         # appending the statistics
#         season_2020.loc[i,'AFT_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
#         season_2020.loc[i,'AFT_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
#         season_2020.loc[i,'AFT_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
#         season_2020.loc[i,'AFT_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
        
        
#         #         calculating off def ratings for team 2
#         tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
#         off_ratings_2 = nba_teams[team_2]['cPTS']/tot_pos_2
#         nba_teams[team_2]['OFFRATE'] = off_ratings_2
        
#         def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
#         nba_teams[team_2]['DEFRATE'] = def_ratings_2
#         # appending the statistics
#         season_2020.loc[i,"OFFRATE_y"] = nba_teams[team_2]['OFFRATE']
#         season_2020.loc[i,'DEFRATE_y'] = nba_teams[team_2]['DEFRATE']   
        
#         #         DISPARITY BETWEEN 2 TEAMS 
#         nba_teams[team_2]['DIS_PTS'] = nba_teams[team_2]['BEF_AVG_PTS'] - nba_teams[team_1]['BEF_AVG_PTS']
#         nba_teams[team_2]['DIS_AST'] = nba_teams[team_2]['BEF_AVG_AST'] - nba_teams[team_1]['BEF_AVG_AST']
#         nba_teams[team_2]['DIS_OREB'] = nba_teams[team_2]['BEF_AVG_OREB'] - nba_teams[team_1]['BEF_AVG_OREB']
#         nba_teams[team_2]['DIS_DREB'] = nba_teams[team_2]['BEF_AVG_DREB'] - nba_teams[team_1]['BEF_AVG_DREB']
#         # appending the statistics
#         season_2020.loc[i,'DIS_PTS_y'] = nba_teams[team_2]['DIS_PTS']
#         season_2020.loc[i,'DIS_AST_y'] = nba_teams[team_2]['DIS_AST']
#         season_2020.loc[i,'DIS_OREB_y'] = nba_teams[team_2]['DIS_OREB']
#         season_2020.loc[i,'DIS_DREB_y'] = nba_teams[team_2]['DIS_DREB']

    

        
        
#     else:
        
#         nba_teams[team_2]['GAME_PLAYED'] += 1
#         season_2020.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED']  
# #         add a before avg ratings for the previous gams 
#         nba_teams[team_2]['BEF_AVG_PTS'] = nba_teams[team_2]['AFT_AVG_PTS']
#         nba_teams[team_2]['BEF_AVG_AST'] = nba_teams[team_2]['AFT_AVG_AST']
#         nba_teams[team_2]['BEF_AVG_OREB'] = nba_teams[team_2]['AFT_AVG_OREB']
#         nba_teams[team_2]['BEF_AVG_DREB'] = nba_teams[team_2]['AFT_AVG_DREB']
#         # appending the statistics
#         season_2020.loc[i,'BEF_AVG_PTS_y'] = nba_teams[team_2]['BEF_AVG_PTS']
#         season_2020.loc[i,'BEF_AVG_AST_y'] = nba_teams[team_2]['BEF_AVG_AST']
#         season_2020.loc[i,'BEF_AVG_OREB_y'] = nba_teams[team_2]['BEF_AVG_OREB']
#         season_2020.loc[i,'BEF_AVG_DREB_y'] = nba_teams[team_2]['BEF_AVG_DREB']
        
    
        
#         nba_teams[team_2]['cPTS'] += row['PTS_y']
#         nba_teams[team_2]['cAST'] += row['AST_y']
#         nba_teams[team_2]['cOREB'] += row['OREB_y']
#         nba_teams[team_2]['cDREB'] += row['DREB_y']
#         nba_teams[team_2]['cFGA'] += row['FGA_y']
#         nba_teams[team_2]['cTO'] += row['TOV_y']
#         nba_teams[team_2]['cFTA'] += row['FTA_y']
#         # appending the statistics
#         season_2020.loc[i,'cPTS_y'] = nba_teams[team_2]['cPTS']
#         season_2020.loc[i,'cAST_y'] = nba_teams[team_2]['cAST']
#         season_2020.loc[i,'cOREB_y'] = nba_teams[team_2]['cOREB']
#         season_2020.loc[i,'cDREB_y'] = nba_teams[team_2]['cDREB']
#         season_2020.loc[i,'cFGA_y'] = nba_teams[team_2]['cFGA']
#         season_2020.loc[i,'cTO_y'] = nba_teams[team_2]['cTO']
#         season_2020.loc[i,'cFTA_y'] = nba_teams[team_2]['cFTA']
        

#         nba_teams[team_2]['AFT_AVG_PTS'] = nba_teams[team_2]['cPTS']/nba_teams[team_2]['GAME_PLAYED']
#         nba_teams[team_2]['AFT_AVG_AST'] = nba_teams[team_2]['cAST']/nba_teams[team_2]['GAME_PLAYED']
#         nba_teams[team_2]['AFT_AVG_OREB'] = nba_teams[team_2]['cOREB']/nba_teams[team_2]['GAME_PLAYED']
#         nba_teams[team_2]['AFT_AVG_DREB'] = nba_teams[team_2]['cDREB']/nba_teams[team_2]['GAME_PLAYED']
#         # appending the statistics
#         season_2020.loc[i,'AFT_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
#         season_2020.loc[i,'AFT_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
#         season_2020.loc[i,'AFT_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
#         season_2020.loc[i,'AFT_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
        
#          #         calculating off def ratings for team 2
#         tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
#         off_ratings_2 = nba_teams[team_2]['cPTS']/tot_pos_2
#         nba_teams[team_2]['OFFRATE'] = off_ratings_2
        
#         def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
#         nba_teams[team_2]['DEFRATE'] = def_ratings_2
#         # appending the statistics
#         season_2020.loc[i,"OFFRATE_y"] = nba_teams[team_2]['OFFRATE']
#         season_2020.loc[i,'DEFRATE_y'] = nba_teams[team_2]['DEFRATE']   
        
#                 #         DISPARITY BETWEEN 2 TEAMS 
#         nba_teams[team_2]['DIS_PTS'] = nba_teams[team_2]['BEF_AVG_PTS'] - nba_teams[team_1]['BEF_AVG_PTS']
#         nba_teams[team_2]['DIS_AST'] = nba_teams[team_2]['BEF_AVG_AST'] - nba_teams[team_1]['BEF_AVG_AST']
#         nba_teams[team_2]['DIS_OREB'] = nba_teams[team_2]['BEF_AVG_OREB'] - nba_teams[team_1]['BEF_AVG_OREB']
#         nba_teams[team_2]['DIS_DREB'] = nba_teams[team_2]['BEF_AVG_DREB'] - nba_teams[team_1]['BEF_AVG_DREB']
#         # appending the statistics
#         season_2020.loc[i,'DIS_PTS_y'] = nba_teams[team_2]['DIS_PTS']
#         season_2020.loc[i,'DIS_AST_y'] = nba_teams[team_2]['DIS_AST']
#         season_2020.loc[i,'DIS_OREB_y'] = nba_teams[team_2]['DIS_OREB']
#         season_2020.loc[i,'DIS_DREB_y'] = nba_teams[team_2]['DIS_DREB']
    
# # # #     add in the data into the dataset 
# #     season_2020.loc[i,'BEF_AVG_PTS_x'] = nba_teams[team_1]['BEF_AVG_PTS']
# #     season_2020.loc[i,'BEF_AVG_PTS_y'] = nba_teams[team_2]['BEF_AVG_PTS']
# #     season_2020.loc[i,'BEF_AVG_AST_x'] = nba_teams[team_1]['BEF_AVG_AST']
# #     season_2020.loc[i,'BEF_AVG_AST_y'] = nba_teams[team_2]['BEF_AVG_AST']
# #     season_2020.loc[i,'BEF_AVG_OREB_x'] = nba_teams[team_1]['BEF_AVG_OREB']
# #     season_2020.loc[i,'BEF_AVG_OREB_y'] = nba_teams[team_2]['BEF_AVG_OREB']
# #     season_2020.loc[i,'BEF_AVG_DREB_x'] = nba_teams[team_1]['BEF_AVG_DREB']
# #     season_2020.loc[i,'BEF_AVG_DREB_y'] = nba_teams[team_2]['BEF_AVG_DREB']
    
# #     season_2020.loc[i,'AFT_AVG_PTS_x'] = nba_teams[team_1]['AFT_AVG_PTS']
# #     season_2020.loc[i,'AFT_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
# #     season_2020.loc[i,'AFT_AVG_AST_x'] = nba_teams[team_1]['AFT_AVG_AST']
# #     season_2020.loc[i,'AFT_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
# #     season_2020.loc[i,'AFT_AVG_OREB_x'] = nba_teams[team_1]['AFT_AVG_OREB']
# #     season_2020.loc[i,'AFT_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
# #     season_2020.loc[i,'AFT_AVG_DREB_x'] = nba_teams[team_1]['AFT_AVG_DREB']
# #     season_2020.loc[i,'AFT_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
    
# #     season_2020.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
# #     season_2020.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED']                                                      
# #     season_2020.loc[i,'cPTS_x'] = nba_teams[team_1]['cPTS']
# #     season_2020.loc[i,'cAST_x'] = nba_teams[team_1]['cAST']
# #     season_2020.loc[i,'cOREB_x'] = nba_teams[team_1]['cOREB']
# #     season_2020.loc[i,'cDREB_x'] = nba_teams[team_1]['cDREB']
# #     season_2020.loc[i,'cFGA_x'] = nba_teams[team_1]['cFGA']
# #     season_2020.loc[i,'cTO_x'] = nba_teams[team_1]['cTO']
# #     season_2020.loc[i,'cFTA_x'] = nba_teams[team_1]['cFTA']
# #     season_2020.loc[i,'cPTS_y'] = nba_teams[team_2]['cPTS']
# #     season_2020.loc[i,'cAST_y'] = nba_teams[team_2]['cAST']
# #     season_2020.loc[i,'cOREB_y'] = nba_teams[team_2]['cOREB']
# #     season_2020.loc[i,'cDREB_y'] = nba_teams[team_2]['cDREB']
# #     season_2020.loc[i,'cFGA_y'] = nba_teams[team_2]['cFGA']
# #     season_2020.loc[i,'cTO_y'] = nba_teams[team_2]['cTO']
# #     season_2020.loc[i,'cFTA_y'] = nba_teams[team_2]['cFTA']

# #     season_2020.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
# #     season_2020.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
# #     season_2020.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
# #     season_2020.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']
# #     season_2020.loc[i,'DIS_PTS_y'] = nba_teams[team_2]['DIS_PTS']
# #     season_2020.loc[i,'DIS_AST_y'] = nba_teams[team_2]['DIS_AST']
# #     season_2020.loc[i,'DIS_OREB_y'] = nba_teams[team_2]['DIS_OREB']
# #     season_2020.loc[i,'DIS_DREB_y'] = nba_teams[team_2]['DIS_DREB']

# #     season_2020.loc[i,"OFFRATE_x"] = nba_teams[team_1]['OFFRATE']
# #     season_2020.loc[i,'DEFRATE_x'] = nba_teams[team_1]['DEFRATE']
# #     season_2020.loc[i,"OFFRATE_y"] = nba_teams[team_2]['OFFRATE']
# #     season_2020.loc[i,'DEFRATE_y'] = nba_teams[team_2]['DEFRATE']                                                            
    

KeyboardInterrupt: 

code test

In [107]:
# for i, row in season_2020.iterrows():
# #         get the team name
#     team_1 = row['TEAM_ABBREVIATION_x']
#     team_2 = row['TEAM_ABBREVIATION_y']

# #     update team 1 and 2
#     if(row['GAME_PLAYED_x'] == 0):
# #         print('hi')
# # #         team 1
#         row["GAME_PLAYED_x"] = 1 
#         nba_teams[team_1]['GAME_PLAYED'] =1
# #         season_2020.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
        
#         row['BEF_AVG_PTS_x'] = 0
#         row['BEF_AVG_AST_x'] = 0
#         row['BEF_AVG_OREB_x'] = 0
#         row['BEF_AVG_DREB_x'] = 0
        
#         nba_teams[team_1]['BEF_AVG_PTS'] = 0
#         nba_teams[team_1]['BEF_AVG_AST'] = 0
#         nba_teams[team_1]['BEF_AVG_OREB'] = 0
#         nba_teams[team_1]['BEF_AVG_DREB'] = 0
# # # appending the statistics        
#         season_2020.loc[i,'BEF_AVG_PTS_x'] =  row['BEF_AVG_PTS_x']
#         season_2020.loc[i,'BEF_AVG_AST_x'] = row['BEF_AVG_AST_x']
#         season_2020.loc[i,'BEF_AVG_OREB_x'] =  row['BEF_AVG_OREB_x']
#         season_2020.loc[i,'BEF_AVG_DREB_x'] = row['BEF_AVG_DREB_x']
   
        
        
#         row['cPTS_x'] += row['PTS_x']
#         row['cAST_x'] += row['AST_x']
#         row['cOREB_x'] += row['OREB_x']
#         row['cDREB_x'] += row['DREB_x']
#         row['cFGA_x'] += row['FGA_x']
#         row['cTO_x'] += row['TOV_x']
#         row['cFTA_x'] += row['FTA_x']
        
#         nba_teams[team_1]['cPTS'] += row['PTS_x']
#         nba_teams[team_1]['cAST'] += row['AST_x']
#         nba_teams[team_1]['cOREB'] += row['OREB_x']
#         nba_teams[team_1]['cDREB'] += row['DREB_x']
#         nba_teams[team_1]['cFGA'] += row['FGA_x']
#         nba_teams[team_1]['cTO'] += row['TOV_x']
#         nba_teams[team_1]['cFTA'] += row['FTA_x']
        
# # #         # appending the statistics
        
#         season_2020.loc[i,'cPTS_x'] = row['cPTS_x']
#         season_2020.loc[i,'cAST_x'] = row['cAST_x']
#         season_2020.loc[i,'cOREB_x'] = row['cOREB_x']
#         season_2020.loc[i,'cDREB_x'] = row['cDREB_x']
#         season_2020.loc[i,'cFGA_x'] = row['cFGA_x'] 
#         season_2020.loc[i,'cTO_x'] = row['cTO_x']
#         season_2020.loc[i,'cFTA_x'] =  row['cFTA_x']
        

#         row['AFT_AVG_PTS_x'] = row['PTS_x']
#         row['AFT_AVG_AST_x'] = row['AST_x']
#         row['AFT_AVG_OREB_X'] = row['OREB_x']
#         row['AFT_AVG_DREB_x'] = row['DREB_x']
        
        
#         nba_teams[team_1]['AFT_AVG_PTS'] = row['AFT_AVG_PTS_x']
#         nba_teams[team_1]['AFT_AVG_AST'] = row['AFT_AVG_AST_x']
#         nba_teams[team_1]['AFT_AVG_OREB'] = row['AFT_AVG_OREB_x']
#         nba_teams[team_1]['AFT_AVG_DREB'] = row['AFT_AVG_DREB_x']
        
#         season_2020.loc[i,'AFT_AVG_PTS_x'] = row['AFT_AVG_PTS_x']
#         season_2020.loc[i,'AFT_AVG_AST_x'] = row['AFT_AVG_AST_x']
#         season_2020.loc[i,'AFT_AVG_OREB_x'] = row['AFT_AVG_OREB_X']
#         season_2020.loc[i,'AFT_AVG_DREB_x'] = row['AFT_AVG_DREB_x']
        

#         #         calculating off def ratings for team 1 
#         tot_pos_1 = nba_teams[team_1]['cFGA'] - nba_teams[team_1]['cOREB'] + nba_teams[team_1]['cTO'] +(0.4* nba_teams[team_1]['cFTA'])
#         off_ratings_1 = nba_teams[team_1]['cPTS']/tot_pos_1
#         nba_teams[team_1]['OFFRATE'] = off_ratings_1
#         row['OFFRATE_x'] = off_ratings_1
        
#         def_ratings_1 = nba_teams[team_2]['cPTS']/tot_pos_1
#         nba_teams[team_1]['DEFRATE'] = def_ratings_1
#         row['DEFRATE_x'] = def_ratings_1
        
#         season_2020.loc[i,"OFFRATE_x"] = row['OFFRATE_x']
#         season_2020.loc[i,'DEFRATE_x'] = row['DEFRATE_x']
#         row['DEFRATE_x'] = def_ratings_1
# # #         DISPARITY BETWEEN 2 TEAMS 
#         row['DIS_PTS_x'] = row['BEF_AVG_PTS_x'] - row['BEF_AVG_PTS_y']
#         row['DIS_AST_x'] = row['BEF_AVG_AST_x'] - row['BEF_AVG_AST_y']
#         row['DIS_OREB_x'] = row['BEF_AVG_OREB_x'] - row['BEF_AVG_OREB_y']
#         row['DIS_DREB_x'] = row['BEF_AVG_DREB_x'] - row['BEF_AVG_DREB_y']
        
#         season_2020.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
#         season_2020.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
#         season_2020.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
#         season_2020.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']

        
#     else:
# # #         add a before avg ratings for the previous gams 

#         row['BEF_AVG_PTS_x'] = nba_teams[team_1]['AFT_AVG_PTS']
#         row['BEF_AVG_AST_x'] = nba_teams[team_1]['AFT_AVG_AST']
#         row['BEF_AVG_OREB_x'] = nba_teams[team_1]['AFT_AVG_OREB']
#         row['BEF_AVG_DREB_x'] = nba_teams[team_1]['AFT_AVG_DREB']
        
#         nba_teams[team_1]['BEF_AVG_PTS'] = row['BEF_AVG_PTS_x']
#         nba_teams[team_1]['BEF_AVG_AST'] = row['BEF_AVG_AST_x']
#         nba_teams[team_1]['BEF_AVG_OREB'] =  row['BEF_AVG_OREB_x']
#         nba_teams[team_1]['BEF_AVG_DREB'] = row['BEF_AVG_DREB_x']

   
#         # appending the statistics
    
#         row["GAME_PLAYED_x"] =  nba_teams[team_1]['GAME_PLAYED'] + 1
#         nba_teams[team_1]['GAME_PLAYED'] = row["GAME_PLAYED_x"]
        
#         row['cPTS_x'] += row['PTS_x']
#         row['cAST_x'] += row['AST_x']
#         row['cOREB_x'] += row['OREB_x']
#         row['cDREB_x'] += row['DREB_x']
#         row['cFGA_x'] += row['FGA_x']
#         row['cTO_x'] += row['TOV_x']
#         row['cFTA_x'] += row['FTA_x']
        
#         nba_teams[team_1]['cPTS'] += row['PTS_x']
#         nba_teams[team_1]['cAST'] += row['AST_x']
#         nba_teams[team_1]['cOREB'] += row['OREB_x']
#         nba_teams[team_1]['cDREB'] += row['DREB_x']
#         nba_teams[team_1]['cFGA'] += row['FGA_x']
#         nba_teams[team_1]['cTO'] += row['TOV_x']
#         nba_teams[team_1]['cFTA'] += row['FTA_x']
        
        
#         row['AFT_AVG_PTS_x'] = ((row['AFT_AVG_PTS_x'] * (row["GAME_PLAYED_x"]-1) )+ row['PTS_x']) /row["GAME_PLAYED_x"]
#         row['AFT_AVG_AST_x'] = ((row['AFT_AVG_AST_x']* (row["GAME_PLAYED_x"]-1) )+ row['AST_x'])/row["GAME_PLAYED_x"]
#         row['AFT_AVG_OREB_x'] = ((row['AFT_AVG_OREB_x']* (row["GAME_PLAYED_x"]-1))+ row['OREB_x'])/row["GAME_PLAYED_x"]
#         row['AFT_AVG_DREB_x'] = ((row['AFT_AVG_DREB_x']* (row["GAME_PLAYED_x"]-1))+ row['DREB_x'])/row["GAME_PLAYED_x"]
        
        
#         nba_teams[team_1]['AFT_AVG_PTS'] = row['AFT_AVG_PTS_x']
#         nba_teams[team_1]['AFT_AVG_AST'] = row['AFT_AVG_AST_x']
#         nba_teams[team_1]['AFT_AVG_OREB'] = row['AFT_AVG_OREB_x']
#         nba_teams[team_1]['AFT_AVG_DREB'] = row['AFT_AVG_DREB_x']
        
# #          #         calculating off def ratings for team 1 
#         tot_pos_1 = nba_teams[team_1]['cFGA'] - nba_teams[team_1]['cOREB'] + nba_teams[team_1]['cTO'] +(0.4* nba_teams[team_1]['cFTA'])
#         off_ratings_1 = nba_teams[team_1]['cPTS']/tot_pos_1
#         nba_teams[team_1]['OFFRATE'] = off_ratings_1
#         row['OFFRATE_x'] = off_ratings_1
        
#         def_ratings_1 = nba_teams[team_2]['cPTS']/tot_pos_1
#         nba_teams[team_1]['DEFRATE'] = def_ratings_1
#         row['DEFRATE_x'] = def_ratings_1

# #         #         DISPARITY BETWEEN 2 TEAMS 
#         row['DIS_PTS_x'] = row['BEF_AVG_PTS_x'] - row['BEF_AVG_PTS_y']
#         row['DIS_AST_x'] = row['BEF_AVG_AST_x'] - row['BEF_AVG_AST_y']
#         row['DIS_OREB_x'] = row['BEF_AVG_OREB_x'] - row['BEF_AVG_OREB_y']
#         row['DIS_DREB_x'] = row['BEF_AVG_DREB_x'] - row['BEF_AVG_DREB_y']
# #         nba_teams[team_1]['DIS_PTS'] = nba_teams[team_1]['BEF_AVG_PTS'] - nba_teams[team_2]['BEF_AVG_PTS']
# #         nba_teams[team_1]['DIS_AST'] = nba_teams[team_1]['BEF_AVG_AST'] - nba_teams[team_2]['BEF_AVG_AST']
# #         nba_teams[team_1]['DIS_OREB'] = nba_teams[team_1]['BEF_AVG_OREB'] - nba_teams[team_2]['BEF_AVG_OREB']
# #         nba_teams[team_1]['DIS_DREB'] = nba_teams[team_1]['BEF_AVG_DREB'] - nba_teams[team_2]['BEF_AVG_DREB']
#         # appending the statistics
#         season_2020.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
#         season_2020.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
#         season_2020.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
#         season_2020.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']
        
#     if(row['GAME_PLAYED_y'] == 0):
#         # #         team 1
#         row["GAME_PLAYED_y"] += 1 
#         nba_teams[team_2]['GAME_PLAYED'] += 1
# #         season_2020.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
        
#         row['BEF_AVG_PTS_y'] = 0
#         row['BEF_AVG_AST_y'] = 0
#         row['BEF_AVG_OREB_y'] = 0
#         row['BEF_AVG_DREB_y'] = 0
        
#         nba_teams[team_2]['BEF_AVG_PTS'] = 0
#         nba_teams[team_2]['BEF_AVG_AST'] = 0
#         nba_teams[team_2]['BEF_AVG_OREB'] = 0
#         nba_teams[team_2]['BEF_AVG_DREB'] = 0
# # # # appending the statistics        
# #         season_2020.loc[i,'BEF_AVG_PTS_x'] = nba_teams[team_1]['BEF_AVG_PTS']
# #         season_2020.loc[i,'BEF_AVG_AST_x'] = nba_teams[team_1]['BEF_AVG_AST']
# #         season_2020.loc[i,'BEF_AVG_OREB_x'] = nba_teams[team_1]['BEF_AVG_OREB']
# #         season_2020.loc[i,'BEF_AVG_DREB_x'] = nba_teams[team_1]['BEF_AVG_DREB']
   
        
        
#         row['cPTS_y'] += row['PTS_y']
#         row['cAST_y'] += row['AST_y']
#         row['cOREB_y'] += row['OREB_y']
#         row['cDREB_y'] += row['DREB_y']
#         row['cFGA_y'] += row['FGA_y']
#         row['cTO_y'] += row['TOV_y']
#         row['cFTA_y'] += row['FTA_y']
        
#         nba_teams[team_2]['cPTS'] += row['PTS_y']
#         nba_teams[team_2]['cAST'] += row['AST_y']
#         nba_teams[team_2]['cOREB'] += row['OREB_y']
#         nba_teams[team_2]['cDREB'] += row['DREB_y']
#         nba_teams[team_2]['cFGA'] += row['FGA_y']
#         nba_teams[team_2]['cTO'] += row['TOV_y']
#         nba_teams[team_2]['cFTA'] += row['FTA_y']
        
# # #         # appending the statistics
        
# #         season_2020.loc[i,'cPTS_x'] = nba_teams[team_1]['cPTS']
# #         season_2020.loc[i,'cAST_x'] = nba_teams[team_1]['cAST']
# #         season_2020.loc[i,'cOREB_x'] = nba_teams[team_1]['cOREB']
# #         season_2020.loc[i,'cDREB_x'] = nba_teams[team_1]['cDREB']
# #         season_2020.loc[i,'cFGA_x'] = nba_teams[team_1]['cFGA']
# #         season_2020.loc[i,'cTO_x'] = nba_teams[team_1]['cTO']
# #         season_2020.loc[i,'cFTA_x'] = nba_teams[team_1]['cFTA']
        

#         row['AFT_AVG_PTS_y'] = row['PTS_y']
#         row['AFT_AVG_AST_y'] = row['AST_y']
#         row['AFT_AVG_OREB_y'] = row['OREB_y']
#         row['AFT_AVG_DREB_y'] = row['DREB_y']
        
        
#         nba_teams[team_2]['AFT_AVG_PTS'] = row['AFT_AVG_PTS_y']
#         nba_teams[team_2]['AFT_AVG_AST'] = row['AFT_AVG_AST_y']
#         nba_teams[team_2]['AFT_AVG_OREB'] = row['AFT_AVG_OREB_y']
#         nba_teams[team_2]['AFT_AVG_DREB'] = row['AFT_AVG_DREB_y']
        

#         #         calculating off def ratings for team 1 
#         tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
#         off_ratings_2 = nba_teams[team_2]['cPTS']/tot_pos_2
#         nba_teams[team_2]['OFFRATE'] = off_ratings_2
#         row['OFFRATE_x'] = off_ratings_2
        
#         def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
#         nba_teams[team_2]['DEFRATE'] = def_ratings_2
#         row['DEFRATE_y'] = def_ratings_2
# # #         DISPARITY BETWEEN 2 TEAMS 
#         row['DIS_PTS_y'] = row['BEF_AVG_PTS_y'] - row['BEF_AVG_PTS_x']
#         row['DIS_AST_y'] = row['BEF_AVG_AST_y'] - row['BEF_AVG_AST_x']
#         row['DIS_OREB_y'] = row['BEF_AVG_OREB_y'] - row['BEF_AVG_OREB_x']
#         row['DIS_DREB_y'] = row['BEF_AVG_DREB_y'] - row['BEF_AVG_DREB_x']

    
#     else:
# # #         add a before avg ratings for the previous gams 

#         row['BEF_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
#         row['BEF_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
#         row['BEF_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
#         row['BEF_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
        
#         nba_teams[team_2]['BEF_AVG_PTS'] = row['BEF_AVG_PTS_y']
#         nba_teams[team_2]['BEF_AVG_AST'] = row['BEF_AVG_AST_y']
#         nba_teams[team_2]['BEF_AVG_OREB'] =  row['BEF_AVG_OREB_y']
#         nba_teams[team_2]['BEF_AVG_DREB'] = row['BEF_AVG_DREB_y']

   
#         # appending the statistics
    
#         row["GAME_PLAYED_y"] =  nba_teams[team_2]['GAME_PLAYED'] + 1
#         nba_teams[team_2]['GAME_PLAYED'] = row["GAME_PLAYED_y"]
        
#         row['cPTS_y'] += row['PTS_y']
#         row['cAST_y'] += row['AST_y']
#         row['cOREB_y'] += row['OREB_y']
#         row['cDREB_y'] += row['DREB_y']
#         row['cFGA_y'] += row['FGA_y']
#         row['cTO_y'] += row['TOV_y']
#         row['cFTA_y'] += row['FTA_y']
        
#         nba_teams[team_2]['cPTS'] += row['PTS_y']
#         nba_teams[team_2]['cAST'] += row['AST_y']
#         nba_teams[team_2]['cOREB'] += row['OREB_y']
#         nba_teams[team_2]['cDREB'] += row['DREB_y']
#         nba_teams[team_2]['cFGA'] += row['FGA_y']
#         nba_teams[team_2]['cTO'] += row['TOV_y']
#         nba_teams[team_2]['cFTA'] += row['FTA_y']
        
        
#         row['AFT_AVG_PTS_y'] = ((row['AFT_AVG_PTS_y']* (row["GAME_PLAYED_y"]-1)+ row['PTS_y']))/row["GAME_PLAYED_y"]
#         row['AFT_AVG_AST_y'] = ((row['AFT_AVG_AST_y']*(row["GAME_PLAYED_y"]-1)+ row['AST_y']) )/row["GAME_PLAYED_y"]
#         row['AFT_AVG_OREB_y'] = ((row['AFT_AVG_OREB_y']*(row["GAME_PLAYED_y"]-1))+ row['OREB_y'])/row["GAME_PLAYED_y"]
#         row['AFT_AVG_DREB_y'] = ((row['AFT_AVG_DREB_y']*(row["GAME_PLAYED_y"]-1)+ row['DREB_y']))/row["GAME_PLAYED_y"]
        
        
#         nba_teams[team_2]['AFT_AVG_PTS'] = row['AFT_AVG_PTS_y']
#         nba_teams[team_2]['AFT_AVG_AST'] = row['AFT_AVG_AST_y']
#         nba_teams[team_2]['AFT_AVG_OREB'] = row['AFT_AVG_OREB_y']
#         nba_teams[team_2]['AFT_AVG_DREB'] = row['AFT_AVG_DREB_y']
        
# #          #         calculating off def ratings for team 1 
#         tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
#         off_ratings_2 = nba_teams[team_2]['cPTS']/ tot_pos_2
#         nba_teams[team_2]['OFFRATE'] = off_ratings_2
#         row['OFFRATE_y'] = off_ratings_2
        
#         def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
#         nba_teams[team_2]['DEFRATE'] = def_ratings_2
#         row['DEFRATE_y'] = def_ratings_2

# #         #         DISPARITY BETWEEN 2 TEAMS 
#         row['DIS_PTS_y'] = row['BEF_AVG_PTS_y'] - row['BEF_AVG_PTS_x']
#         row['DIS_AST_y'] = row['BEF_AVG_AST_y'] - row['BEF_AVG_AST_x']
#         row['DIS_OREB_y'] = row['BEF_AVG_OREB_y'] - row['BEF_AVG_OREB_x']
#         row['DIS_DREB_y'] = row['BEF_AVG_DREB_y'] - row['BEF_AVG_DREB_x']
        
        
# #     else:
        
# #         nba_teams[team_2]['GAME_PLAYED'] += 1
# #         season_2020.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED']  
# # #         add a before avg ratings for the previous gams 
# #         nba_teams[team_2]['BEF_AVG_PTS'] = nba_teams[team_2]['AFT_AVG_PTS']
# #         nba_teams[team_2]['BEF_AVG_AST'] = nba_teams[team_2]['AFT_AVG_AST']
# #         nba_teams[team_2]['BEF_AVG_OREB'] = nba_teams[team_2]['AFT_AVG_OREB']
# #         nba_teams[team_2]['BEF_AVG_DREB'] = nba_teams[team_2]['AFT_AVG_DREB']
# #         # appending the statistics
# #         season_2020.loc[i,'BEF_AVG_PTS_y'] = nba_teams[team_2]['BEF_AVG_PTS']
# #         season_2020.loc[i,'BEF_AVG_AST_y'] = nba_teams[team_2]['BEF_AVG_AST']
# #         season_2020.loc[i,'BEF_AVG_OREB_y'] = nba_teams[team_2]['BEF_AVG_OREB']
# #         season_2020.loc[i,'BEF_AVG_DREB_y'] = nba_teams[team_2]['BEF_AVG_DREB']
        
    
        
# #         nba_teams[team_2]['cPTS'] += row['PTS_y']
# #         nba_teams[team_2]['cAST'] += row['AST_y']
# #         nba_teams[team_2]['cOREB'] += row['OREB_y']
# #         nba_teams[team_2]['cDREB'] += row['DREB_y']
# #         nba_teams[team_2]['cFGA'] += row['FGA_y']
# #         nba_teams[team_2]['cTO'] += row['TOV_y']
# #         nba_teams[team_2]['cFTA'] += row['FTA_y']
# #         # appending the statistics
# #         season_2020.loc[i,'cPTS_y'] = nba_teams[team_2]['cPTS']
# #         season_2020.loc[i,'cAST_y'] = nba_teams[team_2]['cAST']
# #         season_2020.loc[i,'cOREB_y'] = nba_teams[team_2]['cOREB']
# #         season_2020.loc[i,'cDREB_y'] = nba_teams[team_2]['cDREB']
# #         season_2020.loc[i,'cFGA_y'] = nba_teams[team_2]['cFGA']
# #         season_2020.loc[i,'cTO_y'] = nba_teams[team_2]['cTO']
# #         season_2020.loc[i,'cFTA_y'] = nba_teams[team_2]['cFTA']
        

# #         nba_teams[team_2]['AFT_AVG_PTS'] = nba_teams[team_2]['cPTS']/nba_teams[team_2]['GAME_PLAYED']
# #         nba_teams[team_2]['AFT_AVG_AST'] = nba_teams[team_2]['cAST']/nba_teams[team_2]['GAME_PLAYED']
# #         nba_teams[team_2]['AFT_AVG_OREB'] = nba_teams[team_2]['cOREB']/nba_teams[team_2]['GAME_PLAYED']
# #         nba_teams[team_2]['AFT_AVG_DREB'] = nba_teams[team_2]['cDREB']/nba_teams[team_2]['GAME_PLAYED']
# #         # appending the statistics
# #         season_2020.loc[i,'AFT_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
# #         season_2020.loc[i,'AFT_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
# #         season_2020.loc[i,'AFT_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
# #         season_2020.loc[i,'AFT_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
        
# #          #         calculating off def ratings for team 2
# #         tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
# #         off_ratings_2 = nba_teams[team_2]['cPTS']/tot_pos_2
# #         nba_teams[team_2]['OFFRATE'] = off_ratings_2
        
# #         def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
# #         nba_teams[team_2]['DEFRATE'] = def_ratings_2
# #         # appending the statistics
# #         season_2020.loc[i,"OFFRATE_y"] = nba_teams[team_2]['OFFRATE']
# #         season_2020.loc[i,'DEFRATE_y'] = nba_teams[team_2]['DEFRATE']   
        
# #                 #         DISPARITY BETWEEN 2 TEAMS 
# #         nba_teams[team_2]['DIS_PTS'] = nba_teams[team_2]['BEF_AVG_PTS'] - nba_teams[team_1]['BEF_AVG_PTS']
# #         nba_teams[team_2]['DIS_AST'] = nba_teams[team_2]['BEF_AVG_AST'] - nba_teams[team_1]['BEF_AVG_AST']
# #         nba_teams[team_2]['DIS_OREB'] = nba_teams[team_2]['BEF_AVG_OREB'] - nba_teams[team_1]['BEF_AVG_OREB']
# #         nba_teams[team_2]['DIS_DREB'] = nba_teams[team_2]['BEF_AVG_DREB'] - nba_teams[team_1]['BEF_AVG_DREB']
# #         # appending the statistics
# #         season_2020.loc[i,'DIS_PTS_y'] = nba_teams[team_2]['DIS_PTS']
# #         season_2020.loc[i,'DIS_AST_y'] = nba_teams[team_2]['DIS_AST']
# #         season_2020.loc[i,'DIS_OREB_y'] = nba_teams[team_2]['DIS_OREB']
# #         season_2020.loc[i,'DIS_DREB_y'] = nba_teams[team_2]['DIS_DREB']
    
# # # #     add in the data into the dataset 
# #     season_2020.loc[i,'BEF_AVG_PTS_x'] = nba_teams[team_1]['BEF_AVG_PTS']
# #     season_2020.loc[i,'BEF_AVG_PTS_y'] = nba_teams[team_2]['BEF_AVG_PTS']
# #     season_2020.loc[i,'BEF_AVG_AST_x'] = nba_teams[team_1]['BEF_AVG_AST']
# #     season_2020.loc[i,'BEF_AVG_AST_y'] = nba_teams[team_2]['BEF_AVG_AST']
# #     season_2020.loc[i,'BEF_AVG_OREB_x'] = nba_teams[team_1]['BEF_AVG_OREB']
# #     season_2020.loc[i,'BEF_AVG_OREB_y'] = nba_teams[team_2]['BEF_AVG_OREB']
# #     season_2020.loc[i,'BEF_AVG_DREB_x'] = nba_teams[team_1]['BEF_AVG_DREB']
# #     season_2020.loc[i,'BEF_AVG_DREB_y'] = nba_teams[team_2]['BEF_AVG_DREB']
    
# #     season_2020.loc[i,'AFT_AVG_PTS_x'] = nba_teams[team_1]['AFT_AVG_PTS']
# #     season_2020.loc[i,'AFT_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
# #     season_2020.loc[i,'AFT_AVG_AST_x'] = nba_teams[team_1]['AFT_AVG_AST']
# #     season_2020.loc[i,'AFT_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
# #     season_2020.loc[i,'AFT_AVG_OREB_x'] = nba_teams[team_1]['AFT_AVG_OREB']
# #     season_2020.loc[i,'AFT_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
# #     season_2020.loc[i,'AFT_AVG_DREB_x'] = nba_teams[team_1]['AFT_AVG_DREB']
# #     season_2020.loc[i,'AFT_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
    
# #     season_2020.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
# #     season_2020.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED']                                                      
# #     season_2020.loc[i,'cPTS_x'] = nba_teams[team_1]['cPTS']
# #     season_2020.loc[i,'cAST_x'] = nba_teams[team_1]['cAST']
# #     season_2020.loc[i,'cOREB_x'] = nba_teams[team_1]['cOREB']
# #     season_2020.loc[i,'cDREB_x'] = nba_teams[team_1]['cDREB']
# #     season_2020.loc[i,'cFGA_x'] = nba_teams[team_1]['cFGA']
# #     season_2020.loc[i,'cTO_x'] = nba_teams[team_1]['cTO']
# #     season_2020.loc[i,'cFTA_x'] = nba_teams[team_1]['cFTA']
# #     season_2020.loc[i,'cPTS_y'] = nba_teams[team_2]['cPTS']
# #     season_2020.loc[i,'cAST_y'] = nba_teams[team_2]['cAST']
# #     season_2020.loc[i,'cOREB_y'] = nba_teams[team_2]['cOREB']
# #     season_2020.loc[i,'cDREB_y'] = nba_teams[team_2]['cDREB']
# #     season_2020.loc[i,'cFGA_y'] = nba_teams[team_2]['cFGA']
# #     season_2020.loc[i,'cTO_y'] = nba_teams[team_2]['cTO']
# #     season_2020.loc[i,'cFTA_y'] = nba_teams[team_2]['cFTA']

# #     season_2020.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
# #     season_2020.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
# #     season_2020.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
# #     season_2020.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']
# #     season_2020.loc[i,'DIS_PTS_y'] = nba_teams[team_2]['DIS_PTS']
# #     season_2020.loc[i,'DIS_AST_y'] = nba_teams[team_2]['DIS_AST']
# #     season_2020.loc[i,'DIS_OREB_y'] = nba_teams[team_2]['DIS_OREB']
# #     season_2020.loc[i,'DIS_DREB_y'] = nba_teams[team_2]['DIS_DREB']

# #     season_2020.loc[i,"OFFRATE_x"] = nba_teams[team_1]['OFFRATE']
# #     season_2020.loc[i,'DEFRATE_x'] = nba_teams[team_1]['DEFRATE']
# #     season_2020.loc[i,"OFFRATE_y"] = nba_teams[team_2]['OFFRATE']
# #     season_2020.loc[i,'DEFRATE_y'] = nba_teams[team_2]['DEFRATE']                                                            
    

In [108]:
season_2020.to_csv('2020_nba_data.csv',index=False) 

# {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
#             'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
#            'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0
#            "cFTA":0 }